---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler

def blight_model():
    train = pd.read_csv("train.csv", encoding='cp1252', low_memory=False)
    y_train = train['compliance'].fillna(value=0)  # treat null as not paid
    # X_train = train[['ticket_id', 'fine_amount', 'admin_fee',
    #                  'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
    #                  'judgment_amount']].fillna(value=0)
    # X_train = train[['ticket_id', 'fine_amount', 'admin_fee',
    #                  'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
    #                  'judgment_amount']].dropna(axis=1, how='any')
    X_train = train[['ticket_id', 'admin_fee',  # there are NaN in fine_amount only
                     'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
                     'judgment_amount']]

    test = pd.read_csv('test.csv')
    # X_test = test[['ticket_id', 'fine_amount', 'admin_fee',
    #                'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
    #                'judgment_amount']].fillna(value=0)
    X_test = test[['ticket_id', 'admin_fee',
                   'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost',
                   'judgment_amount']]

    # Important!!
    # important to introduce the normalization to deal with verious feature scales
    X_train_scaled = MinMaxScaler().fit_transform(X_train)
    X_test_scaled = MinMaxScaler().fit_transform(X_test)

    # logistic regression
    # For better fit, bigger values specify weaker regularization
    lr = LogisticRegression(C=10).fit(X_train_scaled, y_train)
    train_score = lr.score(X_train_scaled, y_train)
    print(train_score)

    y_proba_lr = lr.predict_proba(X_test_scaled)

    # output the probability that each corresponding ticket will be paid (positive class)
    compliance = pd.Series(y_proba_lr[:, 1], X_test['ticket_id'])

    # # SVM is too expensive
    # svm = SVC().fit(X_train, y_train)
    # train_score = svm.score(X_train, y_train)
    # print(train_score)

    # # Naive Bayes Classifier, Highly efficient but nad generalization
    # nb = GaussianNB().fit(X_train, y_train)
    # train_score = nb.score(X_train, y_train)
    # print(train_score)  # tend to overfit, because features are many
    #
    # y_proba_nb = nb.predict_proba(X_test)
    #
    # # output the probability that each corresponding ticket will be paid (positive class)
    # compliance = pd.Series(y_proba_nb[:, 1], X_test['ticket_id'])

    return compliance


In [2]:
blight_model()

0.956485262039


ticket_id
284932    8.510730e-06
285362    3.900169e-07
285361    1.250282e-05
285338    8.505491e-06
285346    1.250310e-05
285345    8.505401e-06
285347    1.515929e-05
285342    2.657493e-05
285530    3.899176e-07
284989    2.678893e-06
285344    1.515936e-05
285343    3.900282e-07
285340    3.900299e-07
285341    1.515943e-05
285349    1.250305e-05
285348    8.505362e-06
284991    2.678885e-06
285532    2.676688e-06
285406    2.677200e-06
285001    2.678845e-06
285006    3.902276e-07
285405    3.899915e-07
285337    2.677480e-06
285496    1.515586e-05
285497    8.503441e-06
285378    3.900075e-07
285589    2.676457e-06
285585    8.502306e-06
285501    1.250017e-05
285581    3.898874e-07
              ...     
376367    8.901351e-07
376366    6.110569e-06
376362    6.110606e-06
376363    7.408764e-06
376365    8.901378e-07
376364    6.110587e-06
376228    6.111848e-06
376265    6.111505e-06
376286    2.538662e-05
376320    6.110995e-06
376314    6.111051e-06
376327    2.538504e-05
3